## 🤗 Transformers provider a Trainer API to easily train or fine-tune Transformer models

## <div dir="auto">با Tainer در Transformers می‌توان مدل‌های ترنسفورمر را آموزش داد.</div>

###### keywords: Train a transformer model, Train BERT model, آموزش Bert, آموزش ترنسفورمر

##### 1. Here is how we can easily preprocess the GLUE MRPC dataset using dynamic padding.

<div dir="auto">1. در اینجا ما نحوه پیش پردازش داده‌های GLUE MRPC با استفاده از padding داینامیک آورده‌ایم.</div>

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def fnTokenizer(x):
    return tokenizer(x["sentence1"], x["sentence2"], truncation=True)

tokenized_datasets = raw_datasets.map(fnTokenizer, batched = True)
data_collator = DataCollatorWithPadding(tokenizer)

##### 2. We also need a model and some training arguments before creating the Trainer.

<div dir="auto">2. همچنین ما به یک مدل و چند آرگومان (کانفیگ) آموزش قبل از ایجاد Trainer نیاز داریم.</div>

In [ ]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

from transformers import TrainingArguments

training_args = TrainingArguments(
    "test-trainer",
    per_device_eval_batch_size=16,
    per_device_train_batch_size=16,
    num_train_epochs=5,
    learning_rate=2e-5,
    weight_decay=0.01,
)

##### 3. We can then pass everything to the Trainer class and start training.

<div dir="auto">3. سپس می‌توانیم هر آنچه که لازم است را به کلاس Trainer پاس بدهیم و آموزش را شروع کنیم.</div>

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)
trainer.train()

##### 4. The predict method allows us to get the predictions of our model on a whole dataset. We can the use those predictions to compute metrics.

<div dir="auto">4. متد predict به ما این امکان را می دهد که predictهای مدل خود را در کل مجموعه‌داده به دست آوریم. ما می‌توانیم از این پیش‌بینی‌ها برای محاسبه معیارها (سنجش‌ها) استفاده کنیم.</div>

In [ ]:
predictions = trainer.predict(tokenized_datasets["validation"])
print(predictions.predictions.shape, predictions.label_ids.shape)

import numpy as np
from datasets import load_metric

metric = load_metric("glue", "mrpc")
preds = np.argmax(predictions.predictions, axis=-1)
metric.compute(predictions=preds, references=predictions.label_ids)